# Implémentation et distribution sous Spark d'un algorithme d'apprentissage non-supervisé : k-means++

# Importation des librairies nécessaires et mise en place du contexte distribué Spark

In [ ]:
from pyspark import SparkContext, SparkConf
from math import sqrt,log
from random import randint, random, randrange,uniform
from copy import deepcopy

import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

sc = SparkContext.getOrCreate()
sc.stop()

conf = SparkConf().setAppName("kmeans").setMaster("local[*]")
sc = SparkContext(conf=conf)

# Récupération du jeu de données s1.txt, à partir des S-sets : https://cs.joensuu.fi/sipu/datasets/

In [ ]:
s1 = sc.textFile("/Users/sam/s1.txt")

#Nous donnons une structure de liste particulière à notre jeu de données

def extract_split(x):
    splits = x.split('    ')
    return (int(splits[1]), int(splits[2]))

s1 = s1.flatMap(lambda x: x.split('\n')).map(extract_split)

s1 = s1.sample(withReplacement=False,fraction=0.5)

In [ ]:
#Pour visualiser cette structure particulière

s1.takeSample(False, 20)

# Initialisation de l'algorithme k-means++

***Comme nous sommes dans un contexte distribué Spark, nous sommes contraints de redéfinir certaines fonctions, comme par exemple la fonction distance euclidienne (relative à la norme euclidienne usuelle).***

In [ ]:
#Nous ajoutons une nouvelle colonne qui répertoriera les distances entre chaque point et chaque centre.

def compute_distance(xi_indexes, yi_indexes):
    def map(row):
        sum = 0
        for i in range(len(xi_indexes)):
            sum += (row[yi_indexes[i]] - row[xi_indexes[i]]) ** 2
        distance = sqrt(sum)
        return row + (distance,)
    return map

***L'algorithme k-means++ demande de tirer un premier centre de manière uniformément aléatoire parmi l'ensemble des points de données. Pour le reste des centres, il faudra :***

***- calculer la distance entre chaque point xi et le centre déjà établi le plus proche***

***- associer à chaque point xi une probabilité proportionnelle au carré de cette distance (par exemple le carré de cette distance sur la somme des carrés des distances entre chaque point xj et le centre déjà établi le plus proche)***

***- tirer un nouveau centre de manière uniformément aléatoire à l'aide du vecteur de probabilités constitué par l'ensemble des probabilités calculées dans la deuxième étape***

***- réitérer les étapes 2 et 3 jusqu'à ce que k centres aient été choisis***

***- enfin, procéder de la même manière que l'algorithme k-means classique pour ajuster les centres (calculs de barycentres) et repositionner les points qui seraient dans les mauvais clusters***

***Cette approche probabiliste pose problème dans le cadre distribué, nous allons voir pourquoi dans la suite de ce Notebook. Nous allons donc, en plus de la stratégie probabiliste, adopter d'autres stratégies d'initialisation.***

**1ère stratégie : admettons que nous ayons n centres déjà établis. Pour choisir le (n+1)ème centre, nous allons observer les distances de chaque point xi (qui n'est pas un centre déjà établi) aux centres déjà établis. Pour un point donné xi, nous prenons la distance minimale à un centre déjà établi (autrement dit le centre déjà établi le plus proche de xi). Une fois que nous aurons fait ceci pour tous les xi, nous prenons comme (n+1)ème centre le xi qui est le plus éloigné de son centre (déjà établi) le plus proche.**

In [3]:
def reduceMax(dist_indexes):
    def reduce_custom(x1,x2):
        '''
        print("\n")
        '''
        dist_x1 = []
        dist_x2 = []
        for idx in dist_indexes:
            dist_x1.append(x1[idx])
            dist_x2.append(x2[idx])
        
        '''
        print("------")
        
        print("X1 :", x1[0], x1[1])
        print("Dist x1 : ",dist_x1)
        '''
        mindist_x1 =  min(dist_x1)
        
        '''
        print("Min dist x1 :", mindist_x1)
        
        print("X2 :", x2[0], x2[1])
        print("Dist x2 : ", dist_x2)
        '''
        mindist_x2 = min(dist_x2)
        '''
        print("Min dist x2 :", mindist_x2)
        print("------")
        '''
        
        
        result = None
        if(mindist_x1 > mindist_x2):
            result = x1
        else:
            result = x2
        
        '''
        print("Result :", result[0], result[1])
        
        '''
        return result
        
        
    return reduce_custom

**2ème stratégie : admettons que nous ayons n centres déjà établis. Pour choisir le (n+1)ème centre, nous allons observer les distances de chaque point xi (qui n'est pas un centre déjà établi) aux centres déjà établis. Pour un point donné xi, nous sommons ses distances aux n centres déjà établis. Une fois que nous aurons fait ceci pour tous les xi, nous prenons comme (n+1)ème centre le xi dont la somme de ses distances aux n centres déjà établis est la plus grande.**

In [4]:
def reduceMaxSum(dist_indexes):
    def reduce_custom(x1,x2):
        dist_x1 = []
        dist_x2 = []
        for idx in dist_indexes:
            dist_x1.append(x1[idx])
            dist_x2.append(x2[idx])
        
        mindist_x1 =  sum(dist_x1)
        mindist_x2 = sum(dist_x2)
        
        
        if(mindist_x1 > mindist_x2):
            return x1
        else:
            return x2
        
        
    return reduce_custom

**3ème stratégie : il s'agit de faire la même chose que la 2ème stratégie, sauf que pour chaque xi, la somme de ses distances aux n centres déjà établis sera pondérée par l'inverse du nombre de distances calculées pour xi.**

In [5]:
def reduceMaxAverage(dist_indexes):
    def reduce_custom(x1,x2):
        dist_x1 = []
        dist_x2 = []
        for idx in dist_indexes:
            dist_x1.append(x1[idx])
            dist_x2.append(x2[idx])
        
        mindist_x1 =  sum(dist_x1)/len(dist_x1)
        mindist_x2 = sum(dist_x2)/len(dist_x2)
        
        
        if(mindist_x1 > mindist_x2):
            return x1
        else:
            return x2
        
        
    return reduce_custom

**4ème stratégie : il s'agit de la stratégie traditionnelle probabiliste de k-means++**

In [8]:
def mapDistance(dist_indexes):
    def custom_map(row):
        index = row[0]
        table = row[1]
        dists = []
        for idx in dist_indexes:
            dists.append(table[idx])

        min_dist = min(dists)

        return (index, min_dist)
    return custom_map

def basicMaxReduce(x1,x2):
    return max(x1,x2)


def findNextCenter(data, dist_indexes, coord_indexes):
    dataZip = data.zipWithIndex().map(lambda x : (x[1],x[0]))
    print ("Data zip", dataZip.take(3))
    
    #Map Min Dist
    dataZipMap = dataZip.map(mapDistance(dist_indexes))
    print ("Data zip map", dataZipMap.take(3))

    #Normalize distance
    nc = dataZipMap.map(lambda x: x[1])
    
    print ("Data to extract max dist", nc.take(3))
    
    nc = nc.reduce(lambda a,b:a+b)
    
    
    print("Full cumulative sum is  : ", nc)
    
    dataZipMap = dataZipMap.map(lambda x: (x[0], x[1]/nc) )
    
    print("Data zip map normalized ", dataZipMap.take(3))
    
    collection = dataZipMap.collect()
    print(collection[:3])
    
    #Draw best point
    indexes = [item[0] for item in collection]
    prob = [item[1] for item in collection]
    
    print(indexes[:3])
    print(prob[:3])
    print(sum(prob))
    
    cumsum = np.cumsum(prob)
    
    index_best_point = None
    
    draw = uniform(0,1)
    print("Draw : ", draw)
    print("Cumsum 0", cumsum[0])
    if(draw < cumsum[0]):
        index_best_point = indexes[0]
    else:
        current_index = 1
        current_sum = cumsum[current_index]
        previous_sum = cumsum[current_index -1]
        while not(draw < current_sum and draw >= previous_sum):
            #print("Index loop ", current_index)
            #print("Current sum ", current_sum)
            #print("Previous sum", previous_sum)
            
            current_index += 1
            previous_sum = cumsum[current_index -1]
            current_sum = cumsum[current_index]
            
        index_best_point = current_index
    
    best_point = dataZip.filter(lambda x: x[0] == index_best_point).collect()
    print("Best point ", best_point)
    result = tuple([best_point[0][1][coord] for coord in coord_indexes])
    print(result)
    return result

**5ème stratégie : tentative de distribution d'un algorithme probabiliste. L'algorithme est une sorte de tournoi entre les points. Nous avons deux points avec pour chacun  d'entre eux, la distance à leur cluster le plus proche, notées d1 et d2. Nous tirons une loi uniforme sur le compact [0;d1+d2]. Notre règle de décision est la suivante : nous choisissons X1 si U < D1, et nous choisissons X2 sinon. Ainsi, nous réalisons des tournois 1 contre 1 (un tournoi entre deux points), ce qui nous donne de multiples réalisations de lois uniformes en 1 contre 1.**

In [9]:
def reduceMaxWithRandom(dist_indexes):
    def reduce_custom(x1,x2):
        dist_x1 = []
        dist_x2 = []
        for idx in dist_indexes:
            dist_x1.append(x1[idx])
            dist_x2.append(x2[idx])
        
        mindist_x1 =  min(dist_x1)
        mindist_x2 = min(dist_x2)
        
        
        draw = uniform(0,mindist_x1 + mindist_x2)    
        
        '''
        print("\n")
        print("X1 :", x1[0], x1[1])
        print("X2 :", x2[0], x2[1])
        
        print("Min_dist x1 :", mindist_x1)
        print("Min_dist x2 :", mindist_x2)
        print("Draw : ", draw)
        '''
        result = x1
        if(draw > mindist_x1):
            result = x2
            
        #print ("Result : ",result[0], result[1])
        return result
        
            
    return reduce_custom

**6ème stratégie : même stratégie que la 2ème, sauf que notre règle de décision ne sera pas basée sur le maximum entre deux sommes de distances minimales, mais plutôt entre la somme des distances minimales pour X1 et une valeur tirée selon une loi uniforme sur le compact [0 ; somme des distances minimales pour X1 + somme des distances minimales pour X2] (remarque : l'algorithme fait le traitement pour tous les Xi, il ne choisit pas X1 en particulier ; nous avons mentionné X1 ici pour expliquer comment l'algorithme fonctionne sur deux points).**

In [10]:
def reduceMaxWithRandomSum(dist_indexes):
    def reduce_custom(x1,x2):
        dist_x1 = []
        dist_x2 = []
        for idx in dist_indexes:
            dist_x1.append(x1[idx])
            dist_x2.append(x2[idx])
        
        mindist_x1 =  sum(dist_x1)
        mindist_x2 = sum(dist_x2)
        
        
        draw = uniform(0,mindist_x1 + mindist_x2)    
            
        result = x1
        if(draw > mindist_x1):
            result = x2
        return result
        
            
    return reduce_custom

# Les trois algorithmes qui suivent seront utiles à la deuxième phase de l'algorithme k-means++, identique à la deuxième phase de l'algorithme kmeans (une fois l'initialisation terminée)

***Réajustement des centres : calculs des barycentres de chaque cluster***

In [13]:
def compute_average(list_reduce, coord_indexes):
    result = ()
    for idx in coord_indexes:
        coord_list = [reduce_tuple[1][idx] for reduce_tuple in list_reduce]
        average = sum(coord_list)/len(coord_list)
        result += (average,)
    return result

***Représentation graphique des clusters***

In [14]:
def drawGraph(data, cluster_centers):
    datalist = data.collect()
    plt.figure()
    x = [point[0] for point in datalist]
    y = [point[1] for point in datalist]
    
    x_clusters = [point[0] for point in cluster_centers]
    y_clusters = [point[1] for point in cluster_centers]
    
    plt.plot(x,y,'bs', x_clusters, y_clusters, 'r^')

In [15]:
def mapAddTuple(t, coord_idx):
    def map(row):
        
        toAdd = tuple([deepcopy(t[i]) for i in coord_idx])
        
        return row + toAdd
    return map

# Algorithme d'initialisation kmeans++ incluant les algorithmes précédents

In [19]:
def initCluster(data,num_clusters,num_features, reducer, cluster_centers, visualize=False, distributed = True):
    xi_indexes = [idx for idx in range(0,num_features)]
    yi_indexes = [idx for idx in range(num_features, num_features*2)]
    coord_indexes = [idx for idx in range(num_features)]
    dist_indexes = [num_features*2]
    current_clust = 2
    
    '''
    print("Initial xi_indexes : ", xi_indexes)
    print("Initial yi_indexes : ", yi_indexes)
    print("Initial coord_indexes : ",coord_indexes)
    print("Initial dist_indexes : ", dist_indexes)
    print("Intial current_clust :", current_clust)
    print("\n")
    '''
    
    for _ in range(num_clusters-1):
        
        #print("#############################")
        
        
        print("Current cluster points :", cluster_centers)
        '''
        print("Data before adding distance")
        collect = data.collect()
        for line in data.collect():
            print(line, "\n")
        '''
        
        data_dist = data.map(compute_distance(xi_indexes, yi_indexes))
        
        
        
        
        if not distributed:
            new_cluster_point = findNextCenter(coord_indexes=coord_indexes,data=data_dist,dist_indexes=dist_indexes)
        else:
            '''
            print("\n Data with distance")
            for line in data_dist.collect():
                print(line, "\n")

            '''
            reduce_tuple = data_dist.reduce(reducer(dist_indexes=dist_indexes))

            ''' 
            print("\n Data after reduce")
            for line in data_dist.collect():
                print(line, "\n")
            '''
        
        
            new_cluster_point = tuple(reduce_tuple[i] for i in coord_indexes)
            
        cluster_centers.append(new_cluster_point)
        
        
        print("\n Cluster list : ", cluster_centers)
                                  
        #print("Computing center of cluster n° {}".format(current_clust))
        print("\n New cluster point : {}".format(new_cluster_point))
        
        
        
        
#CE MAP ETAIT LA SOURCE DES SOUCI : EN EFFET, NOUS NOUS RETROUVIONS AVEC DES NOUVEAUX CENTRES QUI ECRASAIENT LES
#ANCIENS DANS LE MAP, ET C'EST POUR CELA QUE NOUS NOUS RETROUVIONS AVEC UN NOMBRE DE CENTRES REDUIT (ET DE NOMBREUX
#CENTRES IDENTIQUES) A L'ARRIVEE. AINSI, POUR PALLIER CE PROBLEME, NOUS AVONS ENCAPSULE LE CODE DANS UNE FONCTION.
        
        data_point = data_dist.map(mapAddTuple(new_cluster_point,coord_idx=coord_indexes))
        
        data = data_point
        
        data_point.count()
        
        '''
        print("\n Data after adding new cluster point")
        for line in data_point.collect():
            print(line, "\n")
        '''
        
        if visualize:
            #Assume that data is 2D, and coord is 0,1
            drawGraph(data, cluster_centers)
        
        #Update of variables
        current_clust += 1
        dist_indexes.append(current_clust*num_features + current_clust - 2)
        yi_indexes = [old_value + num_features + 1 for old_value in yi_indexes]
        
        #print("New yi indexes : {}".format(yi_indexes))
        #print("New dist_indexes : {}".format(dist_indexes))
        
        '''
        print("\n")
        print("##############################")
        '''
    return data, cluster_centers

# Exemple d'utilisation de l'algorithme k-means++ sur nos données (version non distribuée et initialisation reduceMax)

In [23]:
cluster1_center = s1.takeSample(False, 1)[0]
cluster_centers = [cluster1_center]
print("Initial cluster center : ",cluster1_center, "\n")
s1_map = s1.map(lambda x : x + cluster1_center)
s1_map_clustered, cluster_centers = initCluster(s1_map, num_features=2,num_clusters=10, reducer=reduceMax, cluster_centers=cluster_centers, visualize=True, distributed=False)


print(cluster_centers)

NameError: name 's1' is not defined

# Exemple d'utilisation de l'algorithme k-means++ sur nos données (version distribuée et initialisation reduceMax)

In [29]:
cluster1_center = s1.takeSample(False, 1)[0]
cluster_centers = [cluster1_center]
print("Initial cluster center : ",cluster1_center, "\n")
s1_map = s1.map(lambda x : x + cluster1_center)
s1_map_clustered, cluster_centers = initCluster(s1_map, num_features=2,num_clusters=10, reducer=reduceMax, cluster_centers=cluster_centers, visualize=True)


print(cluster_centers)

NameError: name 's1' is not defined

# Exemple d'utilisation de l'algorithme k-means++ sur nos données (version distribuée et initialisation reduceMaxWithRandom)

In [30]:
print("Initial cluster center : ",cluster1_center, "\n")
cluster_centers = [cluster1_center]
s1_map_clustered2 = initCluster(s1_map, num_features=2,num_clusters=10, reducer=reduceMaxWithRandom,  cluster_centers=cluster_centers, visualize=True)

NameError: name 'cluster1_center' is not defined

# Exemple d'utilisation de l'algorithme k-means++ sur nos données (version distribuée et initialisation reduceMaxSum)

In [26]:
print("Initial cluster center : ",cluster1_center, "\n")
cluster_centers = [cluster1_center]
s1_map_clustered2 = initCluster(s1_map, num_features=2,num_clusters=10, reducer=reduceMaxSum,  cluster_centers=cluster_centers, visualize=True)

NameError: name 'cluster1_center' is not defined

# Exemple d'utilisation de l'algorithme k-means++ sur nos données (version distribuée et initialisation reduceMaxAverage)

In [31]:
print("Initial cluster center : ",cluster1_center, "\n")
cluster_centers = [cluster1_center]
s1_map_clustered2 = initCluster(s1_map, num_features=2,num_clusters=10, reducer=reduceMaxAverage,  cluster_centers=cluster_centers, visualize=True)

NameError: name 'cluster1_center' is not defined

# Exemple d'utilisation de l'algorithme k-means++ sur nos données (version distribuée et initialisation reduceMaxWithRandomSum)

In [32]:
print("Initial cluster center : ",cluster1_center, "\n")
cluster_centers = [cluster1_center]
s1_map_clustered2 = initCluster(s1_map, num_features=2,num_clusters=10, reducer=reduceMaxWithRandomSum,  cluster_centers=cluster_centers, visualize=True)

NameError: name 'cluster1_center' is not defined

# Algorithme final k-means++ et visualisation toutes les dix itérations

In [33]:
def distance(x1, x2):
    sum = 0
    for i in range(len(x1)):
        sum += (x2[i] - x1[i]) ** 2
    
    distance_result = sqrt(sum)
    
    return distance_result


#L X Y X1 Y1 D1 X2 Y2 D2 X3 Y3 D3
#0 1 2 3  4  5  6  7  8  9  10 11
def computeCluster(num_features, num_clusters):
    def map_cluster(row):
        
        #print("====== \n Map compute cluster distance")
        index_coord = [index for index in range(1,num_features + 1)]
        #print("index coords :", index_coord)
        
        coords_value = [row[index] for index in index_coord]
            
        indexes_clusters = [[index_cluster*num_features + index_features + index_cluster for index_features in range(0,num_features)] for index_cluster in range(1, num_clusters+ 1)]
        #print("index centers initial : ", indexes_clusters)
        
        indexes_distance = [index_cluster*num_features + index_cluster - 1 for index_cluster in range(2, num_clusters+ 2)]
        #print("indexes distance : ", indexes_distance)
        
        cluster_distances = []
        
        for cluster_coord_idx in indexes_clusters :
            
            coords_cluster = [row[index] for index in cluster_coord_idx]
            
            distance_cluster = distance(coords_value, coords_cluster)
            #print("Coord cluster ", coords_cluster)
            #print("Coord value ", coords_value)
            #print("Distance : ", distance_cluster)
            
            cluster_distances.append(distance_cluster)
            
        #print("Cluster distance array ", cluster_distances)
        
        index_max = np.argmin(cluster_distances) + 1
        #print("Index min : ", index_max)
        row_list = list(row)
        row_list[0] = index_max
        
        row = tuple(row_list)
        
        #print("========")
        
        return row
    return map_cluster
    
def formatDataReduce(num_features, num_clusters):
    def map_format(row):
        cluster = row[0]
        #indexes_clusters = [cluster * num_features + index_features + cluster for index_features in range(0,num_features)]
        indexes_coords = [index for index in range(1,num_features + 1)]
        
        coords_value = [row[index] for index in indexes_coords]
        #coords_cluster = [row[index] for index in indexes_clusters]
        
        return (cluster, (coords_value,1))
    return map_format

def reduceComputeCenter(num_features):
    def reduce_center(x1,x2):
        
        #print("X1 ", x1)
        #print("X2", x2)
        
        
        coords_x1 = x1[0]
        coords_x2 = x2[0]
        
        #print("Coords x1 ", coords_x1)
        #print("Coords x2", coords_x2)
        coords_sum = [coords_x1[i] + coords_x2[i] for i in range(0, num_features)]
        
        #print("Sum of coords", coords_sum)
        
        x1_num = x1[1]
        x2_num = x2[1]
        
        return tuple([coords_sum, x1_num + x2_num])
    return reduce_center

def updateClusterCoords(num_features, num_clusters, reduce):
    def map_update_cluster(row):
        
        #print("Reduce : ", reduce)
        indexes_clusters = [[index_cluster*num_features + index_features + index_cluster for index_features in range(0,num_features)] for index_cluster in range(1, num_clusters+ 1)]
        
        #print("index centers : ", indexes_clusters)
        
        num_cluster_real = len(reduce)
        for cluster in range(0,num_cluster_real):
            new_coordinates = reduce[cluster][1]
            cluster_center_indexes = indexes_clusters[cluster]
            
            for i in range(len(cluster_center_indexes)):
                row_list = list(row)
                row_list[cluster_center_indexes[i]] = new_coordinates[i]
                row = tuple(row_list)
        
        return row
            
        
        
    return map_update_cluster

def getRandomColor():
    return '#{:06x}'.format(randint(0, 0xffffff))    
    
def drawKmeans(data, cluster_centers, colorlist):
    datalist = data.collect()
    plt.figure()
    
    dict_point = {}
    
    for point in datalist:
        cluster = point[0]
        pointdict = dict_point.get(cluster, {})
        pointlist = pointdict.get("points",[])
        pointdict["color"] = colorlist[cluster - 1]
        
        pointlist.append([point[1],point[2]])
        pointdict["points"] = pointlist
        
        dict_point[cluster] = pointdict
    
    #print("Dict point ", dict_point)
    
    for key in dict_point:
        pointlist = dict_point[key]["points"]
        color = dict_point[key]["color"]
        
        x = [item[0] for item in pointlist]
        y = [item[1] for item in pointlist]
        
        #print("Color ", color)
        #print("X ", x)
        #print("Y ",y)
        plt.scatter(x,y,color = color, marker="s")
        
    x_clusters = [point[0] for point in cluster_centers]
    y_clusters = [point[1] for point in cluster_centers]
    
    plt.plot(x_clusters, y_clusters, 'r^')
    
def kmeans(data, cluster_centers, num_iterations, num_features, visualize=False):
    if visualize:
        colorlist = [getRandomColor() for i in range(len(cluster_centers))]

    num_clusters = len(cluster_centers)
    #print("Num clusters :", num_clusters, "\n")
    
    #Cluster is first in the tuple
    data = data.map(lambda x : (-1,) + x)
    
    #print("Data with empty cluster", data.take(3), "\n")
    data = data.map(computeCluster(num_features, num_clusters))
    
    #print("Data with cluster", data.take(3), "\n")
    
    for it in range(num_iterations):
        dataReduce = data.map(formatDataReduce(num_features, num_clusters))
        #print("Data prepared for the reduce :", dataReduce.take(3), "\n")
        
        reduce = dataReduce.reduceByKey(reduceComputeCenter(num_features))
        collect = reduce.collect()
        #print("Collect : ", collect)
        
        #COMPUTE AVERAGE
        reduce_list = []
        
        #tuple reduce : ( 1, ([2,3,4], 112))
        for tuple_reduce in reduce.collect():
            cluster = tuple_reduce[0]
            
            tuple_value = tuple_reduce[1]
            
            coord_list = tuple_value[0]
            
            num_point = tuple_value[1]
            
            for i in range(0,num_features):
                coord_list[i] = coord_list[i] / num_point
            
            reduce_list.append( (cluster, coord_list) )
        
        #print("Result of reduce : ", reduce_list, "\n")
        
        
        #reduce list :  (1, [2,3,4])
        data = data.map(updateClusterCoords(num_features, num_clusters, reduce_list))
        #print("Result of map update coords :", data.take(3), "\n")
        
        data = data.map(computeCluster(num_features, num_clusters))
        #print("Data with updated cluster", data.take(3), "\n")
        
        if visualize and (it %10 == 0):
            cluster_centers = []
            for tuple_reduce in reduce_list:
                cluster_centers.append([tuple_reduce[1][0],tuple_reduce[1][1]])
            
        
            drawKmeans(data = data, cluster_centers=cluster_centers, colorlist=colorlist)
             
        print("--------------")

        
    return data

***En utilisant cet algorithme avec une initialisation reduceMax, nous obtenons de bons résultats***

In [36]:
cluster1_center = s1.takeSample(False, 1)[0]
cluster_centers = [cluster1_center]
print("Initial cluster center : ",cluster1_center, "\n")


s1_map = s1.map(lambda x : x + cluster1_center)
s1_map_clustered, cluster_centers = initCluster(s1_map, num_features=2,num_clusters=15, reducer=reduceMax, cluster_centers=cluster_centers, visualize=True)

NameError: name 's1' is not defined

In [35]:
kmeans_result = kmeans(s1_map_clustered, cluster_centers, num_iterations=30, num_features=2, visualize=True)

NameError: name 's1_map_clustered' is not defined

***En utilisant cet algorithme avec une initialisation reduceMaxWithRandom, nous obtenons de moins bons résultats que l'initialisation reduceMax.***

In [37]:
cluster_centers = [cluster1_center]
print("Initial cluster center : ",cluster1_center, "\n")


s1_map = s1.map(lambda x : x + cluster1_center)
s1_map_clustered, cluster_centers = initCluster(s1_map, num_features=2,num_clusters=15, reducer=reduceMaxWithRandom, cluster_centers=cluster_centers, visualize=True)
kmeans_result = kmeans(s1_map_clustered, cluster_centers, num_iterations=30, num_features=2, visualize=True)

NameError: name 'cluster1_center' is not defined

***En Initialisant avec k-means++ non optimisé pour être distribué correctement (mais l'algorithme exact), le résultat est correct mais en théorie il peut mettre du temps à tourner et à nous fournir les résultats finaux.***

In [38]:
cluster_centers = [cluster1_center]
print("Initial cluster center : ",cluster1_center, "\n")


s1_map = s1.map(lambda x : x + cluster1_center)
s1_map_clustered, cluster_centers = initCluster(s1_map, num_features=2,num_clusters=15, reducer=reduceMaxWithRandom, cluster_centers=cluster_centers, distributed=False, visualize=True)
kmeans_result = kmeans(s1_map_clustered, cluster_centers, num_iterations=30, num_features=2, visualize=True)

NameError: name 'cluster1_center' is not defined